##  ewf-ext-03-02-01 - SeaEyes

### <a name="service">Service Definition

In [ ]:
service = dict([('title', 'ewf-ext-03-02-01 - SeaEyes'),
                ('abstract', 'ewf-ext-03-02-01 - SeaEyes'),
                ('id', 'ewf-ext-03-02-01')])

### <a name="parameter">Parameter Definition 

**Adaptive Thresholding**

Adaptive Thresholding: Extend the shoreline by this many pixels (Land-Sea-Mask).

In [ ]:
shorelineExtension = dict([('id', 'shorelineExtension'),
                           ('value', '10'),
                           ('title', 'Shoreline Extension'),
                           ('abstract', 'Shoreline Extension: Extend the shoreline by this many pixels (Land-Sea-Mask)'),
                           ('minOccurs', '1')])

**Probability of false alarm**

Probability of false alarm: The PFA value is computed by 10^(-x). where x is the given positive number (Adaptive Threshold Algorithm).

In [ ]:
pfa = dict([('id', 'pfa'),
            ('value', '12.5'),
            ('title', 'Probability of false alarm'),
            ('abstract', 'Probability of false alarm: The PFA value is computed by 10^(-x). where x is the given positive number (Adaptive Threshold Algorithm)'),
            ('minOccurs', '1')])

**Minimum Target Size (m)**

Minimum Target Size (m): Target with dimension smaller than this threshold is eliminated (Object-Discrimination).

In [ ]:
minTargetSizeInMeter = dict([('id', 'minTargetSizeInMeter'),
                             ('value', '30.0'),
                             ('title', 'Minimum Target Size'),
                             ('abstract', 'Minimum Target Size (m): Target with dimension smaller than this threshold is eliminated (Object-Discrimination).'),
                             ('minOccurs', '1')])

**Maximum Target Size (m)**

Maximum Target Size (m): Target with dimension larger than this threshold is eliminated (Object-Discrimination).

In [ ]:
maxTargetSizeInMeter = dict([('id', 'maxTargetSizeInMeter'),
                             ('value', '600.0'),
                             ('title', 'Maximum Target Size (m)'),
                             ('abstract', 'Maximum Target Size (m): Target with dimension larger than this threshold is eliminated (Object-Discrimination).'),
                             ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

In [ ]:
input_identifiers = ['S1B_IW_GRDH_1SDV_20170703T194823_20170703T194848_006328_00B202_5554']

**Input references**

In [ ]:
input_references = ['https://catalog.terradue.com/sentinel1/search?uid=S1B_IW_GRDH_1SDV_20170703T194823_20170703T194848_006328_00B202_5554']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data/S-1'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
import snappy

import sys
import os

import cioppy

import numpy as np

import gdal

import datetime

import shutil

import pandas as pd
import geopandas as gpd

import shapely.wkt

#### Methods

In [ ]:
import lxml.etree as etree
import subprocess
import tempfile
import time
import psutil
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap


class GraphProcessor():
    
    def __init__(self):
        self.root = etree.Element('graph')
    
        version = etree.SubElement(self.root, 'version')
        version.text = '1.0'
        self.pid = None
        self.p = None
   
    def view_graph(self):
        
        print etree.tostring(self.root , pretty_print=True)
        
    def add_node(self, node_id, operator, parameters, source):
    
        xpath_expr = '/graph/node[@id="%s"]' % node_id

        if len(self.root.xpath(xpath_expr)) != 0:

            node_elem = self.root.xpath(xpath_expr)[0]
            operator_elem = self.root.xpath(xpath_expr + '/operator')[0]
            sources_elem = self.root.xpath(xpath_expr + '/sources')[0]
            parameters_elem = self.root.xpath(xpath_expr + '/parameters')

            for key, value in parameters.iteritems():
                p_elem = self.root.xpath(xpath_expr + '/parameters/%s' % key)[0]
                p_elem.text = value
        else:

            node_elem = etree.SubElement(self.root, 'node')
            operator_elem = etree.SubElement(node_elem, 'operator')
            sources_elem = etree.SubElement(node_elem, 'sources')

            if isinstance(source, list):

                for index, s in enumerate(source):
                    if index == 0:  
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')

                    else: 
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct.%s' % str(index))

                    source_product_elem.attrib['refid'] = s

            elif source != '':
                source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')
                source_product_elem.attrib['refid'] = source

            parameters_elem = etree.SubElement(node_elem, 'parameters')
            parameters_elem.attrib['class'] = 'com.bc.ceres.binding.dom.XppDomElement'

            for key, value in parameters.iteritems():

                parameter_elem = etree.SubElement(parameters_elem, key)
                parameter_elem.text = value

        node_elem.attrib['id'] = node_id

        operator_elem.text = operator 

    def save_graph(self, filename):
        
        with open(filename, 'wb') as file:
            file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
            file.write(etree.tostring(self.root, pretty_print=True))
     
    def plot_graph(self):
        
        for node_id in self.root.xpath('/graph/node/@id'):
            

            xpath_expr = '/graph/node[@id="%s"]' % node_id
            
            if len(self.root.xpath(xpath_expr + '/sources/sourceProduct')) != 0:
                print(self.root.xpath(xpath_expr + '/sources/sourceProduct'))[0].attrib['refid']
                print node_id
            else:
                print node_id
        return True
    
    def run(self):
        
        fd, path = tempfile.mkstemp()
        
        try:
        
            self.save_graph(filename=path)
            
            options = ['/opt/snap6/bin/gpt',
               '-x',
               '-c',
               '2048M',
               path]
            
            options = ['/workspace/temp/temp/snap6/snap6/bin/gpt',
               '-x',
               '-c',
               '2048M',
               path]

            p = subprocess.Popen(options,
                stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)

            print p.pid
            res, err = p.communicate()
            print res, err
            if p.returncode != 0:
                raise Exception('An error occurred during the execution of gpt (see log)')
            
        except Exception as e:
            with open('stdout.txt', 'wb') as file:
                file.write(res)
                file.close()
            with open('stderr.txt', 'wb') as file:
                file.write(err)
                file.close()
            
            raise
        finally:
            os.remove(path)
            
            
def get_operator_default_parameters(operator):
    
    op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

    op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

    #return op_params

    parameters = dict()

    for param in op_params:
    
        #print(param.getName(), param.getDefaultValue())
    
        parameters[param.getName()] = param.getDefaultValue()
        
    return parameters


In [ ]:
def vessel_detection_processing(**kwargs):
   
    options = dict()
    
    operators = ['Read',
                 'Land-Sea-Mask',
                 'Calibration',
                 'AdaptiveThresholding',
                 'Object-Discrimination',
                 'Write']
    
    for operator in operators:
            
        print 'Getting default values for Operator {}'.format(operator)
        parameters = get_operator_default_parameters(operator)
        
        options[operator] = parameters

    for key, value in kwargs.items():
        
        print 'Updating Operator {}'.format(key)
        options[key.replace('_', '-')].update(value)
    
    mygraph = GraphProcessor()
    
    for index, operator in enumerate(operators):
    
        print 'Adding Operator {} to graph'.format(operator)
        if index == 0:            
            source_node_id = ''
        
        else:
            source_node_id = operators[index - 1]
        
        mygraph.add_node(operator,
                         operator, 
                         options[operator], source_node_id)
    
    mygraph.view_graph()
    
    mygraph.run()

#### Aux folders

In [ ]:
output_folder = ''
temp_folder = 'temp'

In [ ]:
if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)
        
if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Operators definition

In [ ]:
reads = []
for s1path in input_identifiers:
    
    read = dict()

    s1meta = "manifest.safe"
    
    s1prd = "%s/%s/%s.SAFE/%s" % (data_path, s1path, s1path, s1meta)
    
    read['file'] =  s1prd
    #read['formatName'] = 'Sen3_SLSTRL1B_500m'
    
    reads.append(read)
    


In [ ]:
landseamask = get_operator_default_parameters('Land-Sea-Mask')

for p in landseamask:
    if p == 'shorelineExtension':
        landseamask[p] = shorelineExtension['value']

landseamask

In [ ]:
calibration = get_operator_default_parameters('Calibration')

#for p in LandSeaMask:
#    if p == 'shorelineExtension':
#        LandSeaMask[p] = '10'

calibration

In [ ]:
adaptivethresholding = get_operator_default_parameters('AdaptiveThresholding')

for p in adaptivethresholding:
    if p == 'pfa':
        adaptivethresholding[p] = pfa['value']

adaptivethresholding

In [ ]:
objectdiscrimination = get_operator_default_parameters('Object-Discrimination')

for p in objectdiscrimination:
    if p == 'minTargetSizeInMeter':
        objectdiscrimination[p] = minTargetSizeInMeter['value']
    elif p == 'maxTargetSizeInMeter':
        objectdiscrimination[p] = maxTargetSizeInMeter['value']

objectdiscrimination

In [ ]:
writes = []
for s1path in input_identifiers:
    
    write = dict()
    
    output_path = os.path.join(temp_folder, s1path + '_temp')
    
    write['file'] = output_path

    writes.append(write)
writes

#### Processing

In [ ]:
for r,w in zip(reads,writes):
    
    vessel_detection_processing(Read=r,
                                Land_Sea_Mask=landseamask,
                                Calibration=calibration,
                                AdaptiveThresholding=adaptivethresholding,
                                Object_Discrimination=objectdiscrimination,
                                Write=w)


#### AIS

##### Detected vessels

In [ ]:
shipdetections_path = os.path.join(writes[0]['file'] + '.data', 'vector_data', 'ShipDetections.csv')
shipdetections_path

In [ ]:
# to pd dataframe
shipdetections_raw = pd.read_csv(shipdetections_path,sep='\t', skiprows=[0])
shipdetections_raw.head()

In [ ]:
# clean and add geometry
shipdetections_gdf = gpd.GeoDataFrame(shipdetections_raw[['ShipDetections', 'Detected_width:Double', 'Detected_length:Double']], geometry=gpd.points_from_xy(shipdetections_raw['Detected_lon:Double'], shipdetections_raw['Detected_lat:Double']), crs="EPSG:4326")

shipdetections_gdf.head()

In [ ]:
shipdetections_gdf.to_file("result2.shp")

##### Read AIS data

In [ ]:
ais_data = '/workspace/Better_3rd_phase/Applications/EXT-03-02-01/ewf-ext-03-02-01/src/main/app-resources/notebook/etc/ais_2017_07.csv'

In [ ]:
# Import AIS data (first to DataFrame, then convert to GeoDataFrame).
ais_df = pd.read_csv(ais_data, decimal=",", usecols=['MMSI', 'IMO', 'STATUS', 'SPEED(KNOTS x10)', 'LON', 'LAT',
                                                     'COURSE', 'HEADING', 'TIMESTAMP(UTC)'],
                     dtype={'LON': float, 'LAT': float})

ais_df['TIMESTAMP(UTC)'] = pd.to_datetime(ais_df['TIMESTAMP(UTC)'])

print('Number of rows in %s: %d' % (ais_data.split('\\')[-1], len(ais_df.index)))



# add geometry
ais_gdf = gpd.GeoDataFrame(ais_df, geometry=gpd.points_from_xy(ais_df.LON, ais_df.LAT), crs="EPSG:4326")
ais_gdf.head()

In [ ]:
# filter by AOI
AOI = shapely.wkt.loads('POLYGON ((-29.97968971227660973 39.49718800212976078, -26.71593511606884519 39.49718800212976078, -26.75288328130893234 37.42809074868483776, -29.99200576735664114 37.46503891392492847, -29.97968971227660973 39.49718800212976078))')
ais_gdf_aoi = ais_gdf[ais_gdf['geometry'].within(AOI)]
ais_gdf_aoi.head()

In [ ]:
# filter in time
ais_gdf_aoi = ais_gdf_aoi[(ais_gdf_aoi['TIMESTAMP(UTC)'] > '2017-07-3 19:40:00') & (ais_gdf_aoi['TIMESTAMP(UTC)'] < '2017-07-3 19:50:00')]

In [ ]:
len(ais_gdf_aoi)

In [ ]:
# first way
sorted = ais_gdf_aoi.sort_values(['MMSI', 'TIMESTAMP(UTC)'], ascending = [True, False])

first = sorted.groupby('MMSI').first().reset_index()

res = gpd.GeoDataFrame(first)

res.head()

In [ ]:
len(res)

In [ ]:
res.to_file("ais.shp")

#### Remove temporary folder

In [ ]:
shutil.rmtree(temp_folder)

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.